In [1]:
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.
For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities
Compatible with: spaCy v2.0.0+
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy

import os, sys, shlex, subprocess
from random import randint
import pandas as pd

In [2]:
#Función de lectura de pdf.
def Read_pdf(file, Original = True ,Complete = False, init_page=0, final_page=1):    
    #Lectura de solo una parte del pdf
    if not Complete:
        init_page = str(init_page) #pagina inicial
        final_page= str(final_page) #pagina final
        if not Original: #Extraer texto sin estructura
            pdftotext = 'pdftotext -f '+init_page+' -l '+final_page+' '
        else: #Documento con estructura original (maintain original physical layout)
            pdftotext = 'pdftotext -layout -f '+init_page+' -l '+final_page+' '
    #Lectura del pdf completo 
    else:
        if not Original: #Extraer texto sin estructura
            pdftotext = 'pdftotext '
        else: #Documento con estructura original
            pdftotext = 'pdftotext -layout '

    command_line = pdftotext+file+'.pdf '+file+'.txt' 
    args = shlex.split(command_line)
    proceso = subprocess.Popen(args, 
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE)
    errores = proceso.stderr.read()
    #Mensajes de error en la conversión de pdf a texto plano
    if errores != b'':
        proceso.stderr.close()
        errores = errores.decode(sys.getdefaultencoding())
        print(errores)
    else:
        txt = file+'.txt'
        archivo = open(txt, 'r') 
        text = archivo.read() 
        archivo.close() 
        return text

In [3]:
local = '1701.05345'
#local = 'Ibarra'
a = True
i = 0
Dir = True
while a:    
    i+=1
    page_content = Read_pdf(local,False ,False ,i ,i)
    #page_content = page_content.title()
    if 'References' in page_content:        
        page_content = Read_pdf(local,False ,False ,i ,-1)        
        a = False
    

val = page_content.find('References')
page_content = page_content[val+len('References'):]

a = True
j = i-1

while a:    
    k = page_content.find('– '+str(j)+' –')
    if k != -1:
        page_content = page_content.replace('– '+str(j)+' –', '')
        j+=1
    else:
        a = False
page_content = page_content.replace('\n\n\n', '')    

a = True
i = 1

while a:    
    k = page_content.find('['+str(i)+']')
    if k != -1:
        page_content = page_content.replace('['+str(i)+']', '~')
        i+=1
    else:
        a = False
page_content = page_content.replace('\n', '')        
page_content = page_content.split(sep='~')

print(page_content)

['', ' S. Weinberg, Baryon- and Lepton-Nonconserving Processes, Phys. Rev. Lett. 43 (1979)1566–1570.', ' P. Minkowski, µ → eγ at a rate of one out of 109 muon decays?, Phys. Lett. B67 (1977)421–428.', ' T. Yanagida, Horizontal gauge symmetry and masses of neutrinos, in Proceedings of theWorkshop on The Unified Theory and the Baryon Number in the Universe (O. Sawada andA. Sugamoto, eds.), p. 95, KEK, Tsukuba, Japan, 1979.', ' M. Gell-Mann, P. Ramond and R. Slansky, Complex spinors and unified theories, inSupergravity (P. van Nieuwenhuizen and D. Z. Freedman, eds.), p. 315, North Holland,Amsterdam, 1979.', ' S. L. Glashow, The future of elementary particle physics, in Proceedings of the 1979 CargèseSummer Institute on Quarks and Leptons (M. Lévy, J.-L. Basdevant, D. Speiser, J. Weyers,R. Gastmans and M. Jacob, eds.), pp. 687–713, Plenum Press, New York, 1980.', ' R. N. Mohapatra and G. Senjanović, Neutrino Mass and Spontaneous ParityNonconservation, Phys. Rev. Lett. 44 (1980) 912–915.

In [4]:
# training data 
TRAIN_DATA = [
    (' J. F. Gunion and H. E. Haber, CP -conserving two-Higgs-doublet model: The Approach tothe decoupling limit, Phys. Rev. D67 (2003) 075019, [hep-ph/0207010].',{
        'entities': [(1, 14, 'AUTOR'), (18, 29, 'AUTOR'), (31, 106, 'TITLE'), (108, 136, 'REV'), (139, 153, 'COD')]
    }),
    (' S. Weinberg, Baryon- and Lepton-Nonconserving Processes, Phys. Rev. Lett. 43 (1979)1566–1570.',{
        'entities': [(1, 12, 'AUTOR'), (14, 56, 'TITLE'), (58, 93, 'REV')]
    }),
    (' S. L. Glashow, The future of elementary particle physics, in Proceedings of the 1979 CargèseSummer Institute on Quarks and Leptons (M. Lévy, J.-L. Basdevant, D. Speiser, J. Weyers,R. Gastmans and M. Jacob, eds.), pp. 687–713, Plenum Press, New York, 1980.',{
        'entities': [(1, 14, 'AUTOR'), (16, 213, 'TITLE'), (216, 227, 'REV')]
    }),
    (' S. Davidson, C. Peña-Garay, N. Rius and A. Santamaria, Present and future bounds onnon-standard neutrino interactions, JHEP 03 (2003) 011, [hep-ph/0302093].',{
        'entities': [(1, 12, 'AUTOR'), (14, 28, 'AUTOR'),(30, 37, 'AUTOR'),(42, 55, 'AUTOR'),(57, 119, 'TITLE'),(121, 138, 'REV'),(142, 156, 'COD')]
    }),
    (' F. Feroz, M. Hobson and M. Bridges, MultiNest: an efficient and robust Bayesianinference tool for cosmology and particle physics, Mon. Not. Roy. Astron. Soc. 398 (2009)1601–1614, [0809.3437].',{
        'entities': [(1, 9, 'AUTOR'),(11, 20, 'AUTOR'),(24, 35, 'AUTOR'),(37, 129, 'TITLE'),(131, 178, 'REV'),(181, 190, 'COD')]
    }),
    (' G. Passarino and M. J. G. Veltman, One-loop corrections for e+ e− annihilation into µ+ µ−in the Weinberg model, Nucl. Phys. B160 (1979) 151–207.\x0c',{
        'entities': [(1, 13, 'AUTOR'),(17, 34, 'AUTOR'),(36, 111, 'TITLE'),(113, 144, 'REV')]
    }),
    (' A. Y. Smirnov and Z.-j. Tao, Neutrinos with Zee mass matrix in vacuum and matter, Nucl.Phys. B426 (1994) 415–433, [hep-ph/9403311].',{
        'entities': [(1, 15, 'AUTOR'),(19, 28, 'AUTOR'),(30, 81, 'TITLE'),(83, 113, 'REV'),(116, 130, 'COD')]
    }),
    (' V. M. Lobashev, The search for the neutrino mass by direct method in the tritium beta-decayand perspectives of study it in the project KATRIN, Nucl. Phys. A719 (2003) C153–C160.',{
        'entities': [(1, 15, 'AUTOR'),(17, 142, 'TITLE'),(144, 177, 'REV')]
    }),
    (' SINDRUM II collaboration, W. H. Bertl et al., A search for µ-e conversion in muonicgold, Eur. Phys. J. C47 (2006) 337–346.',{
        'entities': [(1, 25, 'AUTOR'),(27, 38, 'AUTOR'),(47, 87, 'TITLE'),(89, 122, 'REV')]
    }),
    (' ACME collaboration, J. Baron et al., Order of Magnitude Smaller Limit on the ElectricDipole Moment of the Electron, Science 343 (2014) 269–272, [1310.7534].',{
        'entities': [(1, 19, 'AUTOR'),(21, 29, 'AUTOR'),(38, 115, 'TITLE'),(117, 143, 'REV'),(146, 155, 'COD')]
    }),
    (' H. Bélusca-Maı̈to and A. Falkowski, On the exotic Higgs decays in effective field theory,Eur. Phys. J. C76 (2016) 514, [1602.02645].',{
        'entities': [(1, 19, 'AUTOR'),(23, 36, 'AUTOR'),(38, 90, 'TITLE'),(91, 119, 'REV'),(122, 132, 'COD')]
    }),
    (' Gfitter Group collaboration, M. Baak, J. Cúth, J. Haller, A. Hoecker, R. Kogler,K. Mönig et al., The global electroweak fit at NNLO and prospects for the LHC and ILC,Eur. Phys. J. C74 (2014) 3046, [1407.3792].',{
        'entities': [(1, 28, 'AUTOR'),(30, 37, 'AUTOR'),(39, 47, 'AUTOR'),(49, 58, 'AUTOR'),(60, 70, 'AUTOR'),(72, 81, 'AUTOR'),(82, 92, 'AUTOR'),(100, 168, 'TITLE'),(169, 198, 'REV'),(201, 210, 'COD')]
    }),
    (' EXO-200 collaboration, J. B. Albert et al., Search for Majorana neutrinos with the firsttwo years of EXO-200 data, Nature 510 (2014) 229–234, [1402.6956].',{
        'entities': [(1, 22, 'AUTOR'),(24, 36, 'AUTOR'),(45, 114, 'TITLE'),(116, 141, 'REV'),(144, 153, 'COD')]
    }),
    (' CMS collaboration, S. Chatrchyan et al., Observation of a new boson at a mass of 125 GeVwith the CMS experiment at the LHC, Phys. Lett. B716 (2012) 30–61, [1207.7235].',{
        'entities': [(1, 18, 'AUTOR'),(20, 33, 'AUTOR'),(42, 123, 'TITLE'),(125, 154, 'REV'),(157, 166, 'COD')]
    }),
    (' A. Ghosal, Y. Koide and H. Fusaoka, Lepton flavor violating Z decays in the Zee model,Phys. Rev. D64 (2001) 053012, [hep-ph/0104104].',{
        'entities': [(1, 10, 'AUTOR'),(12, 20, 'AUTOR'),(24, 35, 'AUTOR'),(37, 86, 'TITLE'),(88, 115, 'REV'),(118, 132, 'COD')]
    }),
    (' F. Capozzi, E. Lisi, A. Marrone, D. Montanino and A. Palazzo, Neutrino masses andmixings: Status of known and unknown 3ν parameters, Nucl. Phys. B908 (2016) 218–234,[1601.07777].',{
        'entities': [(1, 11, 'AUTOR'), (13, 20, 'AUTOR'),(22, 32, 'AUTOR'),(34, 46, 'AUTOR'),(51, 61, 'AUTOR'),(63, 132, 'TITLE'),(134, 165, 'REV'),(167, 177, 'COD')]
    }),
    (' Y. Koide, Prospect of the Zee model, Nucl. Phys. B Proc. Suppl. 111 (2002) 294–296,[hep-ph/0201250].',{
        'entities': [(1, 9, 'AUTOR'),(11, 36, 'TITLE'),(38, 83, 'REV'),(85, 99, 'COD')]
    }),
    (' S. Davidson, C. Peña-Garay, N. Rius and A. Santamaria, Present and future bounds onnon-standard neutrino interactions, JHEP 03 (2003) 011, [hep-ph/0302093].',{
     'entities': [(1, 12, 'AUTOR'),(14, 28, 'AUTOR'), (30, 37, 'AUTOR'), (42, 55, 'AUTOR'), (57, 119, 'TITLE'), (121, 139, 'REV'), (142, 156, 'COD')]
    }),
    (' R. N. Mohapatra and G. Senjanović, Neutrino Mass and Spontaneous ParityNonconservation, Phys. Rev. Lett. 44 (1980) 912–915.',{
        'entities': [(1, 16, 'AUTOR'), (21, 34, 'AUTOR'), (37, 88, 'TITLE'), (90, 124, 'REV')]
    }),
    (' F. Feroz, M. Hobson, E. Cameron and A. Pettitt, Importance Nested Sampling and theMultiNest Algorithm, 1306.2144.',{
        'entities': [(1, 9, 'AUTOR'), (11, 20, 'AUTOR'), (22, 32, 'AUTOR'), (37, 47, 'AUTOR'), (49, 102, 'TITLE'), (104, 113, 'COD')]
    })
]

In [ ]:
n_iter = 100
nlp = spacy.blank('en')  # create blank Language class
print("Created blank 'en' model")
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        #print(losses)

In [ ]:
        
#test the trained model
#TEST_DATA = ['Who is Shakira Pique?',  'I live in Medellín.', 'Is Tony Star famous?',  'I like Boston.' \
#             , 'Quién es Roberto Bolaños?', 'Me gusta Buenos Aires.']
#for text in TEST_DATA:
#    doc = nlp(text)
#    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
TEST = 'K. Balaji, W. Grimus and T. Schwetz, The solar LMA neutrino oscillation solution in theZee model, Phys. Lett. B508 (2001) 301–310, [hep-ph/0104035]'
doc = nlp(TEST)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])    

In [ ]:
TEST = ' Daya Bay collaboration, F. P. An et al., Observation of Electron-AntineutrinoDisappearance at Daya Bay, Phys. Rev. Lett. 108 (2012) 171803, [1203.1669].'
doc = nlp(TEST)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
# save model to output directory
output_dir = './Model'
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

In [5]:
# test the saved model
output_dir = './Model'
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

#TEST = ' Daya Bay collaboration, F. P. An et al., Observation of Electron-AntineutrinoDisappearance at Daya Bay, Phys. Rev. Lett. 108 (2012) 171803, [1203.1669].'
TEST = ' F. Feroz, M. Hobson, E. Cameron and A. Pettitt, Importance Nested Sampling and theMultiNest Algorithm, 1306.2144.'
doc = nlp2(TEST)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Loading from ./Model
Entities [('F. Feroz', 'AUTOR'), ('M. Hobson', 'AUTOR'), ('E. Cameron', 'AUTOR'), ('A. Pettitt', 'AUTOR'), ('Importance Nested Sampling and theMultiNest Algorithm', 'TITLE'), ('1306.2144', 'COD')]


In [6]:
doc

 F. Feroz, M. Hobson, E. Cameron and A. Pettitt, Importance Nested Sampling and theMultiNest Algorithm, 1306.2144.

In [7]:
doc.ents[5].text_with_ws

'1306.2144'

In [8]:
AUTOR = []
TITLE = None
REV = None
COD = None
j=5
print(page_content[j])
doc = nlp2(page_content[j])

for i in range(len(doc.ents)):
    if doc.ents[i].label_ == 'AUTOR':
        AUTOR.append(doc.ents[i].text)
        i+=1
    elif doc.ents[i].label_ == 'TITLE':
        TITLE = doc.ents[i].text
        i+=1
    elif doc.ents[i].label_ == 'REV':
        REV = doc.ents[i].text
        i+=1    
    elif doc.ents[i].label_ == 'COD':
        COD = doc.ents[i].text
        i+=1        
print(AUTOR)
print(TITLE)
print(REV)
print(COD)

 S. L. Glashow, The future of elementary particle physics, in Proceedings of the 1979 CargèseSummer Institute on Quarks and Leptons (M. Lévy, J.-L. Basdevant, D. Speiser, J. Weyers,R. Gastmans and M. Jacob, eds.), pp. 687–713, Plenum Press, New York, 1980.
['S. L. Glashow']
The future of elementary particle physics, in Proceedings of the 1979 CargèseSummer Institute on Quarks and Leptons (M. Lévy, J.-L. Basdevant, D. Speiser, J. Weyers,R. Gastmans and M. Jacob, eds.
pp. 687–713
None


In [9]:
df = pd.DataFrame()
O=pd.Series({"authors": AUTOR,
            "title": TITLE,
            "journal": REV,
            "COD": COD,
            })
df=df.append(O,ignore_index=True)

In [10]:
df

,COD,authors,journal,title
0,None,[S. L. Glashow],pp. 687–713,"The future of elementary particle physics, in ..."


In [11]:
page_content[1]

' S. Weinberg, Baryon- and Lepton-Nonconserving Processes, Phys. Rev. Lett. 43 (1979)1566–1570.'

In [12]:
df = pd.DataFrame()

for j in range(1,len(page_content)):
    AUTOR = []
    TITLE = None
    REV = None
    COD = None

    doc = nlp2(page_content[j])

    for i in range(len(doc.ents)):
        if doc.ents[i].label_ == 'AUTOR':
            AUTOR.append(doc.ents[i].text)
            i+=1
        elif doc.ents[i].label_ == 'TITLE':
            TITLE = doc.ents[i].text
            i+=1
        elif doc.ents[i].label_ == 'REV':
            REV = doc.ents[i].text
            i+=1    
        elif doc.ents[i].label_ == 'COD':
            COD = doc.ents[i].text
            i+=1            
    O=pd.Series({"authors": AUTOR, "title": TITLE, "journal": REV, "COD": COD})
    df=df.append(O,ignore_index=True)

In [13]:
df

,COD,authors,journal,title
0,None,[S. Weinberg],Phys. Rev. Lett. 43 (1979)1566–1570,Baryon- and Lepton-Nonconserving Processes
1,None,[P. Minkowski],Phys. Lett. B67 (1977)421–428,µ → eγ at a rate of one out of 109 muon decays?
2,None,[T. Yanagida],1979.,"Tsukuba, Japan"
3,None,"[M. Gell-Mann, P. Ramond, R. Slansky]","Amsterdam, 1979",North Holland
4,None,[S. L. Glashow],pp. 687–713,"The future of elementary particle physics, in ..."
5,None,"[R. N. Mohapatra, G. Senjanović]",Phys. Rev. Lett. 44 (1980) 912–915,Neutrino Mass and Spontaneous ParityNonconserv...
6,1207.7214,"[ATLAS collaboration, G. Aad]",Phys. Lett. B716 (2012)1–29,Observation of a new particle in the search fo...
7,1207.7235,"[CMS collaboration, S. Chatrchyan]",Phys. Lett. B716 (2012) 30–61,Observation of a new boson at a mass of 125 Ge...
8,1502.07400,"[CMS collaboration, V. Khachatryan]",Phys. Lett. B749 (2015) 337–362,Search for lepton-flavour-violating decays of ...
9,None,"[ATLAS collaboration, G. Aad]",1604.07730.,Search for lepton-flavour-violating decays of ...
